# Bag of Words (BoW)

단어의 등장순서를 고려하지 않고 빈도에 기반한 벡터 표현


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 450kB 48.3MB/s 
     |████████████████████████████████| 92kB 14.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt # 형태소 분석기
import re 
okt = Okt()

In [ ]:
token = re.sub('(W.)', "", "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")
# 정규표현식을통해 온점을 제거하는 작업이다

token = okt.morphs(token)
# okt 형태소 분석기를 통해 토큰화작업을 수행한 뒤 toke 에 넣어둔다

word2index = {}
bow = []

for voca in token :
  if voca not in word2index.keys() :
    word2index[voca] = len(word2index)
    # token을 읽으면서, word2index에 없는 (not in)단어는 새로 추가하고, 이미 있는 단어는 넘김
    bow.insert(len(word2index)-1, 1)
    # bow 전체에 전부 기본값 1을 넣어준다, 단어의 갯수는 최소 1개 이상이니까
  else :
    index = word2index.get(voca)
    # 재등장하는 단어의 인덱스 받기
    bow[index] = bow[index]+1
    # 재등장하는 단어는 해당하는 인덱스에 1 더하자
print(word2index)

{'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10, '.': 11}


In [ ]:
# 만든 bow
bow

[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]

In [ ]:
# keras에서 bow를 쓸 수 있다.

## keras Tokenizer를 활용한 BoW

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ['John likes to watch movies. Mary likes movies too! Mary also likes to watch football games.']

In [ ]:
# 함수 불러오자!
def print_bow(sentence) :
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentence) # 단어장 생성
  bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow 에 저장

  print('Bag of Words :', bow) # bow 출력
  print('단어장(vocabulary)의 크기:', len(tokenizer.word_counts)) # 중복 제거한 단어수

## scitit-learn의 CountVectorizer를 이용한 BoW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
sentence = ['John likes to watch movies. Mary likes movies too! Mary also likes to watch football games.']


vector = CountVectorizer() # 빈도수 표현

print("Bag of Words :", vector.fit_transform(sentence).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록
print('각 단어의 인덱스:', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되는지

Bag of Words : [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스: {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


### 불용어를 제거한 BoW 만들기

정확도를 높이기 위함



#### 사용자가 직접 정의한 불용어 사용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text =["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=['the', 'a', 'an', 'is', 'not'])
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


#### CountVectorizer에서 제공하는 자체 불용어

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text =["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words='english')
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


#### NLTK에서 지원하는 불용어 사용

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text =["Family is not an important thing. It's everything."]
sw = stopwords.words('english')
vect = CountVectorizer(stop_words=sw)
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


In [ ]:
# nltk가 더 좋은거 같다고 말씀하심

# DTW (Document_Term Matrix)

다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것

다수의 문서에 대해서 BoW를 하나의 행렬로 표현하고 부르는 용어

In [ ]:
import pandas as pd
content=[[0, 1, 1, 1], [1, 0, 1, 1], [2, 0, 2, 2]]
df = pd.DataFrame(content)
df.index = ['(문서1) I like dog', '(문서2) I like cat', '(문서3) I like dog I like cat']
df.columns = ['cat', 'dog', 'I', 'like']
df

,cat,dog,I,like
(문서1) I like dog,0,1,1,1
(문서2) I like cat,1,0,1,1
(문서3) I like dog I like cat,2,0,2,2


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

def cos_sim(A, B) :
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


### sciktit-learn CountVectorizer를 활용하여 DTM구현

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
          "John likes to watch movies",
          "Mary likes movies too",
          "Mary also likes to watch football games too"
]

vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 1 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


한계점
1. 희소표현(sparse)
2. 단순 빈도수 기반 접근

##  TF-IDF(Term Frequency-Inverse Document Frequency)

In [ ]:
from math import log
import pandas as pd
import numpy as np

docs = [
          "John likes to watch movies and Mary likes movies too",
          "james likes to watch TV",
          "Mary also likes to watch football games too"
]


In [ ]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'james', 'likes', 'movies', 'to', 'too', 'watch']


In [ ]:
N = len(docs)

In [ ]:
def tf(t, d) :
  return d.count(t)

def idf(t) :
  df = 0
  for doc in docs :
    df+= t in doc
    return log(N/(df + 1)) +1

def tfidf(t, d) : 
  return tf(t,d) * idf(t)

In [ ]:
result = []

for i in range(N) :
  result.append([])
  d = docs[i]
  for j in range(len(vocab)) : 
    t = vocab[j]

    result[-1].append(tf(t,d))

tf_= pd.DataFrame(result, columns = vocab)
tf_

,John,Mary,TV,also,and,football,games,james,likes,movies,to,too,watch
0,1,1,0,0,1,0,0,0,2,2,2,1,1
1,0,0,1,0,0,0,0,1,1,0,1,0,1
2,0,1,0,1,0,1,1,0,1,0,2,1,1


In [ ]:
result = []
for j in range(len(vocab)) :
  t = vocab[j]
  result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])
idf_

,IDF
John,1.405465
Mary,1.405465
TV,2.098612
also,2.098612
and,1.405465
football,2.098612
games,2.098612
james,2.098612
likes,1.405465
movies,1.405465


In [ ]:
result=[]
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns=vocab)
tfidf_

,John,Mary,TV,also,and,football,games,james,likes,movies,to,too,watch
0,1.405465,1.405465,0.000000,0.000000,1.405465,0.000000,0.000000,0.000000,2.810930,2.81093,2.810930,1.405465,1.405465
1,0.000000,0.000000,2.098612,0.000000,0.000000,0.000000,0.000000,2.098612,1.405465,0.00000,1.405465,0.000000,1.405465
2,0.000000,1.405465,0.000000,2.098612,0.000000,2.098612,2.098612,0.000000,1.405465,0.00000,2.810930,1.405465,1.405465


### scikit-learn을 활용한 TF-IDF구현

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
          'you know I want your love',
          'I like you',
          'what should I do'
]

vector = CountVectorizer()

In [ ]:
print(vector.fit_transform(corpus).toarray())

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
          'you know I want your love',
          'I like you',
          'what should I do'
]

tfidfv = TfidfVectorizer().fit(corpus)

In [ ]:
print(tfidfv.transform(corpus).toarray())

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]


In [ ]:
print(tfidfv.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### abc뉴스 데이터로 tf-idf

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv",
                           filename="/content/abcnews-data-text.csv")


('/content/abcnews-data-text.csv', <http.client.HTTPMessage at 0x7fbf81ad6190>)

In [ ]:
data = pd.read_csv('/content/abcnews-data-text.csv', error_bad_lines=False)

In [ ]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
text=data[['headline_text']]

In [ ]:
text.nunique() # 고유한 값을 출력

headline_text    1054983
dtype: int64

In [ ]:
text.drop_duplicates(inplace=True)
text = text.reset_index(drop=True)
print(len(text))

1054983


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


데이터 정제 및 정규화

In [ ]:
# nltk 토크나이저로 토큰화 하기
text['headline_text'] = text.apply(lambda row : nltk.word_tokenize(row['headline_text']), axis = 1)

In [ ]:
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x : [word for word in x if word not in (stop_words)])

In [ ]:
text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [ ]:
# 이제 정규화

In [ ]:
text['headline_text'] = text['headline_text'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

In [ ]:
text = text['headline_text'].apply(lambda x : [word for word in x if len(word) > 2])

In [ ]:
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [ ]:
detokenized_doc = []
for i in range(len(text)) :
  t = ' '.join(text[i])
  detokenized_doc.append(t)


train_data = detokenized_doc

In [ ]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [ ]:
c_vectorizer = CountVectorizer(stop_words = 'english', max_features=5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

In [ ]:
print('행렬의 크기 :', document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

In [ ]:
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


# 실제 뉴스기사 크롤링 및 분류

In [ ]:
!pip install beautifulsoup4
!pip install konlpy

  Using cached https://files.pythonhosted.org/packages/d7/b9/51afecb35bb61b188a4b44868001de348a0e8134b4dfa00ffc191567c4b9/newspaper3k-0.2.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/82/1251fefec3bb4b03fd966c7e7f7a41c9fc2bb00d823a34c13f847fd61406/feedfinder2-0.0.4.tar.gz
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 92kB 13.3MB/s 
     |████████████████████████████████| 7.4MB 40.0MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=c6c5e97c99baf59b191edaf3ba4edc6a04f6fb83201a1986cb88930d12694a4f
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398404 sha256=44d73eb7abe2305ad820dae149f3c8a782f6d036aa70adde0d5e5283b05adace
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34


In [ ]:
!pip install newspaper3k

     |████████████████████████████████| 215kB 31.1MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 7.4MB 28.9MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=a4a2714ef260d6148fa970d9a970343459b2f85411ea8ffce14decb52f39becf
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398404 sha256=de6549a0f5cdbbc3cf5790af982e567400eea3789eb5a28bba3a8a47402a2429
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13553 sha256=7fc131e73d253ba5a607548b8264baec4f74ef3ffc2a670d893d3b540733bb5e
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [ ]:
cd Mecab-ko-for-Google-Colab/

[Errno 2] No such file or directory: 'Mecab-ko-for-Google-Colab/'
/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 153kB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 450kB 48.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-07-08 05:12:35--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c3:9b0a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=%2BPrhxMRbNdlU9LwVfASsUQmAFa0%3D&Expires=1625722082&AWSAccessKeyId=AKIA6KOSE3

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

## Beautifulsoup 사용법

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# 임의로 만든 html
html = '''
<html>
  <head>
  </head>
  <body>
    <h1>장바구니
      <p id='clothes' class = 'name' title = '라운드티'>라운드티
        <span class = 'number'> 25 </span>
        <span class = 'price'> 29000 </span>
        <span class = 'menu'> 의류 </span>
        <a href = 'http://www.naver.com'> 바로가기 </a>
      </p>
      <p id ='watch' class='name' title='시계'> 시계
        <span class='number'> 28 </span>
        <span class='price'> 32000 </span>
        <span class='menu'> 악세서리 </span>
        <a href = 'http://www.facebook.com'> 바로가기 </a>
      </p>
    </h1>
  </body>
  </html>
  '''

In [ ]:
soup = BeautifulSoup(html, 'html.parser') # 분석할 파서의 종류를 지정하고 객체 생성

In [ ]:
print(soup.select('body'))

[<body>
<h1>장바구니
      <p class="name" id="clothes" title="라운드티">라운드티
        <span class="number"> 25 </span>
<span class="price"> 29000 </span>
<span class="menu"> 의류 </span>
<a href="http://www.naver.com"> 바로가기 </a>
</p>
<p class="name" id="watch" title="시계"> 시계
        <span class="number"> 28 </span>
<span class="price"> 32000 </span>
<span class="menu"> 악세서리 </span>
<a href="http://www.facebook.com"> 바로가기 </a>
</p>
</h1>
</body>]


In [ ]:
print(soup.select('p'))

[<p class="name" id="clothes" title="라운드티">라운드티
        <span class="number"> 25 </span>
<span class="price"> 29000 </span>
<span class="menu"> 의류 </span>
<a href="http://www.naver.com"> 바로가기 </a>
</p>, <p class="name" id="watch" title="시계"> 시계
        <span class="number"> 28 </span>
<span class="price"> 32000 </span>
<span class="menu"> 악세서리 </span>
<a href="http://www.facebook.com"> 바로가기 </a>
</p>]


In [ ]:
print(soup.select('h1 .name .menu'))

[<span class="menu"> 의류 </span>, <span class="menu"> 악세서리 </span>]


In [ ]:
from newspaper import Article

In [ ]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002881076'

In [ ]:
article = Article(url, language='ko')
article.download()
article.parse()

In [ ]:
print('기사 제목 :')
print(article.title)

기사 제목 :
[AI 사피엔스 시대]자연어처리 기술, 컴퓨팅 파워 경쟁 시대로


In [ ]:
print('기사 내용 :')
print(article.text)

기사 내용 :
[Copyright ⓒ 전자신문 & 전자신문인터넷, 무단전재 및 재배포 금지]

주로 아이디어와 기술력으로 경쟁했던 자연어처리 인공지능(AI) 분야는 점차 컴퓨팅 파워 싸움으로 무게 추가 이동하고 있다. 모델이 대형화되면서 향상된 퍼포먼스 확보에 필요한 자금 규모도 커지고 있다. 자칫 대기업 자본력에 휘둘릴 수 있다는 우려도 함께 나온다.자연어처리(NLP)는 인간이 사용하는 언어 체계를 기계가 인식하도록 알고리즘을 디자인하는 기술이다. 흔히 말하는 컴퓨터 혹은 인간과 대화하는 컴퓨터 관련 기술이 포함된다.목적에 따라 세 가지 카테고리로 나뉜다. 인간이 제기한 질문에 자동으로 적절한 답을 찾아주는 '질의응답(QA)', 원하는 업무를 지시했을 때 작업을 수행하는 '테스크 컴플리션', 그리고 특별한 목적이 없는 대화를 의미하는 '오픈도메인 컨버세이션(비목적성 대화)'이 있다. 각기 발전해왔던 세 가지 기술은 지난 2018년 구글의 인공지능 언어모델 '버트(BERT)'의 등장으로 패러다임이 전환됐다. 압도적인 성능으로 대량의 프리트레이닝(사전학습)이 가능해지면서 굳이 셋을 구분할 필요가 없어진 것이다.기계학습 연구에서 모델을 학습할 때는 지도학습과 비지도학습, 강화학습 중 하나를 골라 활용한다. 지도학습은 사람이 적절한 입력과 출력을 부여하는 방식이다. 정답이 정해져 있고 기계의 정답률도 쉽게 측정할 수 있다. 반면에 비지도학습은 정답이 정해지지 않은 데이터에 대해서도 기계가 스스로 클러스터링 등을 통해 학습한다. 체계화되지 않은 대량의 데이터를 학습 가능하지만 학습이 맞게 됐는지 확인하기 어렵다.버트는 기존 AI 학습 방법을 혁신적으로 바꿔놨다는 평가를 받는다. 자연어처리를 교사 없이 양방향으로 사전 학습하는 최초의 시스템이다. 비지도학습 방식을 사용하면서도 기존 존재했던 어떤 기술보다 뛰어난 성능을 보여준다. 최근 1년 반 동안 버트를 필두로 AI 모델은 급격히 대형화되는 추세다.이는 기존의 빅데이터 개념을 훨씬 상회하는 초 대규모 데이터를 프리트레

### BeautifulSoup와 newspaper3k를 이용해 크롤링

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def make_urllist(page_num, code, date):
  urllist = []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=' + str(code) + '&date=' + str(date) + '&page' +str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)

    soup = BeautifulSoup(news.content, 'html.parser')

    # case1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # case2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))

    for line in news_list:
      urllist.append(line.a.get('href'))
      # 각 뉴스로부터 a 태그인 <a href='주소'>에서 '주소'만을 가져옵니다.
  return urllist

In [ ]:
url_list = make_urllist(2, 101, 20200506)

In [ ]:
print('뉴스 기사의 갯수 :', len(url_list))

뉴스 기사의 갯수 : 40


In [ ]:
# 그 날짜의 기사들 리스트
url_list[:5]

['https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451723',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451721',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451718',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0009849190',
 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=057&aid=0001451717']

In [ ]:
idx2word = {'101' : '경제', 
            '102' : '사회', 
            '103' : '생활/문화',
            '105' : 'IT/과학'}

In [ ]:
# 데이터프레임으로 만들어주는 함수를 만들자
from newspaper import Article

def make_data(urllist, code) : 
  text_list = []
  for url in urllist :
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  df = pd.DataFrame({'news' : text_list})

  df['code'] = idx2word[str(code)]

  return df

In [ ]:
data = make_data(url_list, 101)
data[:10]

,news,code
0,고려은단이 5월을 맞아 응원 메시지를 공유하는 ‘5월 5글자로 응원 부탁해!’ 이벤...,경제
1,코리아나화장품의 민감성 피부를 위한 저자극 스킨케어 브랜드 '프리엔제'가 마르고 건...,경제
2,서울장수주식회사가 부드럽고 달콤한 맛으로 인기를 모으고 있는 생막걸리 ‘인생막걸리’...,경제
3,[서울=뉴시스] 오동현 기자 = 모바일 게임 기업 컴투스는 3D 모바일 야구 게임 ...,경제
4,대원제약이 2020년 상반기 신입과 경력 정기 공채를 실시합니다.정기 공채 모집분야...,경제
5,"[AFP=연합뉴스] [AFP=연합뉴스]\n\n""요즘은 잔인한 날""…리프트도 앞서 9...",경제
6,이재용 삼성전자 부회장이 6일 삼성전자 서울 서초사옥에서 대국민 사과 회견을 하기 ...,경제
7,JW중외제약이 A형 혈우병 예방요법제 ‘헴리브라피하주사를 출시하고 본격적인 마케팅 ...,경제
8,"옵티팜과 휴벳바이오가 공동 개발중인 백신 후보 물질에 대해 마우스, 기니피그, 미니...",경제
9,[한국경제TV 신동호 기자]\n\n전남 나주시와 충북 청주시가 방사광 가속기 구축사...,경제


### 데이터 수집 및 전처리

In [ ]:
code_list = [102, 103, 105]

In [ ]:
def make_total_data(page_num, code_list, date) : 
  df = None
  
  for code in code_list :
    url_list = make_urllist(page_num, code, date)
    df_temp = make_data(url_list, code)
    print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

    if df is not None :
      df = pd.concat([df, df_temp])
    else :
      df = df_temp

  return df

In [ ]:
df = make_total_data(1, code_list, 20200506)

102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


In [ ]:
print('# of article :', len(df))

# of article : 60


In [ ]:
df.sample(10)

,news,code
6,[서울=뉴시스] 오동현 기자 = 펍지주식회사가 올해 첫 '플레이어언노운스 배틀그라운...,IT/과학
0,[서울=뉴시스] 오동현 기자 = 모바일 게임 기업 컴투스는 3D 모바일 야구 게임 ...,IT/과학
7,동영상 뉴스\n\n5월 날씨가 맞나 싶으시죠.오늘도 초여름이었습니다.심지어 올 들어...,생활/문화
13,[KBS 전주][앵커]해마다 이맘때면 전주 영화의 거리에는 봄의 영화 축제를 즐기려...,생활/문화
4,지난 2016년 포항공대에 구축한 4세대 선형 방사광가속기. /연합뉴스 지난 201...,IT/과학
5,질서정연 코로나19 확산 방지를 위한 ‘물리적 거리 두기’가 ‘생활 속 거리 두기’...,사회
2,나는 지난 30여년간 ‘협력(協力)’이라는 말을 매일 마주하며 살고 있다. 남북관계...,생활/문화
12,동영상 뉴스\n\n[앵커]다음 주 고등학교 3학년부터 순차적인 등교수업이 시작되는데...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
8,"6일 등교수업을 앞둔 경북 한 학교의 보건실에는 손소독제, 마스크 등 방역물품이 상...",사회


In [ ]:
# 지금까지는 1페이지에 대해서만 크롤링을 해봤는데 이번엔 많이 해보자

In [ ]:
df = make_total_data(100, code_list, 20200506) # 경고 : 이거 한시간걸림 # 에이 실패했네

102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.


ArticleException: ignored

In [ ]:
# 강사님 올려주신거로 다시해보자

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
pwd

'/content/Mecab-ko-for-Google-Colab'

In [77]:
csv_path = '/content/drive/MyDrive/Colab Notebooks/news_data.csv'
df = pd.read_table(csv_path, sep=',')
df.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


In [78]:
df['news'] = df['news'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]", "")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
5995    서울경제 넷플릭스의 망 무임승차와 텔레그램의 불법 촬영물 유통을 막는 법안이 국회 ...
5996    일 국회에서 열린 과방위 법안소위 일 국회에서 열린 과방위 법안소위아이뉴스 민혜정 ...
5997    디지털데일리 이종현기자 공공시설을 이용하거나 공공기관 운영 강좌 수강을 신청할 때 ...
5998    기사 섹션 분류 안내기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다 언론사...
5999    넷플릭스와 유튜브 페이스북 등에게 국내 이용자를 위한 서비스 안정성을 유지할 책임을...
Name: news, Length: 6000, dtype: object

In [79]:
len(df)

6000

In [82]:
print(df.isnull().sum())

news    0
code    0
dtype: int64


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 4019
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   news    53 non-null     object
 1   code    53 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [81]:
df.drop_duplicates(subset=['news'], inplace=True)
print('뉴스기사의 갯수 :', len(df))
# 음... 뭔가 잘못된거같은데? 

뉴스기사의 갯수 : 53


In [85]:
# 다른거 쓰자. 
csv_path = '/content/drive/MyDrive/Colab Notebooks/news_data(1).csv'
df = pd.read_table(csv_path, sep=',')
df.head()

FileNotFoundError: ignored